# Homework 4

## Objective: Get the lowest loss on a CIFAR10 Classifier

After taking the entirety of the Deep Learning course, I have had the opportunity to learn various of techniques to solve different problems using classical deep learning techniques. However, for this homework, I decided that the two most important things that will be helpful for me to get a lower loss on the CIFAR image classification task are Batch/Layer Normalization and ResNets. As such, I first decided to focus on making the best model I got in homework 2 better by introducing normalization and also trying different model architectures to lower the loss. Finally, I will showcase the ResNet I made to see if it can beat the optimized model from homework 2.

### Route 1: Optimization of HW2 Model

#### Initial Model Architecture

To start, I decided to take a look at the model from the last homework that worked best for this task. Before doing anything, this model had 4 CNN layers with channels (30 -> 64 -> 128 -> 256), kernel sizes (5x5 -> 3x3 -> 3x3 -> 3x3), and a constant stride of 1, a Max Pooling layer with kernel 2x2 and stride 2, and finally a classification head with linear layers (1000 -> 500 -> 250 -> 10). Of course, dropout was placed between every layer when possible. Looking at the model, I saw that it was not currently using any form of normalization, so I thought to add batch normalization between the CNN layers and layer normalization between the linear layers. Furthermore, I decided to do this not just because it was a new tool but also since I saw that it was effective when used for the models in Homework 3.

#### Model Training

For the first training loop, I decided to use the hyperparameters that were kept for the original model in Homework 2 since the new architecture is very similar to the old architecture. For reference the hyperparameters used to train the new model are...

---

batch_size = 64

learning_rate = 5e-4

decay_rate = 4e-4

c_dropout = 0.30

f_dropout = 0.30

---

After training the model, I noticed large improvements in training and testing loss being 0.6250 and 0.6327 at the 20th epoch which is better than the ~0.80 training and testing loss for the architecture from homework 2. From here, I tried to tinker with learning rate by setting it to 1e-4 but the model not only took longer to get to an optimal training and testing loss, around the 35th epoch, but could only get a loss around ~0.64. Fiddling with the dropout was also not successful and would only make the model take longer to train but not get losses lower than ~0.70.

#### Second Model Architecture(s)

After exhausting all of my options for the first model I made, I decided to instead try and make the model more complex, I noticed that the CNN Layers would expand the number of channels in a reverse funnel, but I never thought to just try and keep the channels constant. Originally, I used a reverse funnel since I wanted the model to be able to get a few base features in the first layer and then use those to create more and more complex features in subseqeunt layers. However, if I tried keeping the same, I could achieve the same effect but instead of tying to just repeatetly getting more and more complex features, I would be building more and more complex features in a more controlled way. As such, I decided to keep the number of channels to 64 for all four CNN layers.

So, so the architecture for this model is now...

4x CNN layers (64 -> 64 -> 64 -> 64) with kernels 3x3 and stride 1 into a Max Pool of kernel 2x2 and stride 2 finally into a Linear Classifier (1000 -> 500 -> 250 -> 10).

Also you might notice that this section is suffixed with "Architecture(s)." This is because I experimented with various constant sizes for the CNN Layers which will be explained in the next section

#### Second Model Training

Once again, I decided to keep the hyperparameters that were previously listed since they seem to be the most optimal for the current model. By the 62th epoch, the model was able to get a new training and testing loss of ~0.60 around the 60th epoch. Knowing that it would be best to keep the hyperparameters the same, I decided that the only way forward would be to just adjust the width of the CNN layers, so I decided to go from 64 channels per layer to 128 channels per layer which allowed me to get a lower training and testing loss of ~0.57. For my next change, I decided to go even further and try using a constant size of 256 for the CNN channels and got a testing and training loss ~0.50 at the 31th epoch. From this, we can see that increasing the channels in the CNN layers in this way allows the model to reach better and better losses for the most part, but of course, this comes at the cost of longer training times. For one final push, I tried increasing the channels to 512 but not only did it take a while to train the lowest loss it got was around ~0.54 at the 17th epoch.

#### Results

Refer to Figure 1 for the loss graph.

---

Final Accuracy: 82.89%

Final Training Loss: ~0.50

Final Testing Loss: ~0.50

Best Model Architecture: 4x CNN Layers (256 -> 256 -> 256 -> 256) of kernels 3x3 and stride 1 into MaxPool Layer of kernel 2x2 and stride 2 into Linear Classifier with layers (1000 -> 500 -> 250 -> 100 -> 10)

---

batch_size = 64

learning_rate = 1e-4

decay_rate = 4e-4

c_dropout = 0.30

f_dropout = 0.30

### Route 2: ResNet Classifier

#### ResNet Design

To start, I decided that I wanted to make ResNet blocks out of purely CNN layers. This is because the original ResNet paper which used a ResNet on image classification also used a similar architecture, so I wanted to see if making a NN like it would be benefitital. To start, I decided to define a CNN block with two CNN layers that keep constant planes (another terminology meaning channels) based on user specification. The block keeps the size of the out planes constant so that we can perform the math of adding the logits processed from the block back to the original input. In addition, there are batch normalization layers in between the CNN layers. For the final ResNet, I made it so that it can make stages with a number of blocks per stages as specified by user. For the first stage, the NN keeps the dimensionality of the image the same, however, subsequent stages will downsample the image. After all the stages, the output is given to a Linear Classifier with dropout layers.

#### Initial Model Architecture

In the beginning, I decided to make the NN start with three stages each with 3 blocks. Additionally, I decided to start the model with 32 planes since I wanted a baseline to see how well the model can perform. Lastly, the Linear Classifier was made of four layers of sizes (2000 -> 1500 -> 1000 -> 500) with a final classification layer of size 10 and had dropout.

For this trial, it was mearly a test run to see where I stood and what could be improved.

#### Initial Model Training

To start, I decided to use the hyperparameters...

---

batch_size = 64

learning_rate = 1e-3

decay_rate = 1e-4

f_dropout = 0.5

---

I wanted to keep use a higher learning rate and dropout as a baseline to see where I needed to adjust. Though the weight decay is also higher, I chose this not as a baseline but rather as another measure to further combat any overfitting since the model is not using dropout in the CNN blocks or layer normalization in the Linear Classifier. After training, the model was able to get a training loss of ~0.45 and testing loss of ~0.58 at the 16th epoch. Afterwards, the model would continue to overtrain severally. Though I wasn't able to immediately beat the best loss of the model in first half of the homework, the model is close to it without immediate tuning. Speaking of tuning, I decided against tuning the hyperparameters for this model since I wanted to test how long I can make the NN and beat the best model.

#### Second Model Architecture

For the next model, I decided to increase the number of stages from 3 to 4 and test changing the time image downsampling occurs. Instead of downsampling after the first stage, this model downsamples after the second stage since I wanted to find the bottleneck where the model could no longer benefit from keeping constant planes. For instance, for a simple regression task using linear layers, there is a point where adding more and more layers does not help model perform any better. Lastly, I decided to keep the starting in planes to 16 since I wanted to be able to train the model efficiently and increased the width of the Linear Classifier to see if there would be any benefit.

As such, the model architecture is now...

4x Stages (3 blocks per stage), starting with 16 planes and subsequence planes having 16 more planes than the last and downsampling starting at the third stage, into a Linear Classifier with 4x layers (2500 -> 1500 -> 1000 -> 500) into final output layer

#### Second Model Training

Again, I kept the hyperparameters stated previously. After training, the model was able to get a training loss of ~0.50 and testing loss of ~0.58 at the 17th epoch, beating the last ResNet. Training past this epoch caused the testing loss to oscillate between ~0.57 and ~0.63 as the training loss kept on decreasing. I tried experimenting with layer normalization for the ResNet, however, using it over dropout caused the ResNet to struggle to get loss lower than 1.0 during training.

#### Final Testing and ResNet Conclusions

In my last exploratory attempt, I decided to see the effects of increasing the in planes of the model with the stages 3 and 4 and saw no visible improvements. For instance, I tried to start the in planes at 64 or 128 with each subsequent stage having double the last stage in terms of planes, however, they could never really pass the ~0.57 testing loss barrier and would be stuck oscillating after the 25th epoch.

Though the Second Model Architecture was able to barely outperform the first ResNet model, I believe that it was simply due to chance rather than being a better architecture. In fact, since the first ResNet is actually smaller in terms of overall parameters, I would say that outperforms the second ResNet model.

#### Best Results for ResNet

Refer to Figure 2 for the loss graph.

---

Final Accuracy: 80.93%

Final Training Loss: ~0.45

Final Testing Loss: ~0.58

Best Model Architecture: 3x Stages (3 blocks per stage), starting with 32 planes and subsequence planes having 32 more planes than the last and downsampling starting at the second stage, into a Linear Classifier with 5x layers (2000 -> 1500 -> 1000 -> 500 -> 10) into final output layer

---

batch_size = 64

learning_rate = 1e-3

decay_rate = 5e-4

linear_dropout = 0.55

---

#### Conclusion

After my tests, I can conclude that the CIFAR10 classification task prefers NN's that are wider instead of longer which can be seen with the standard CNN outperforming the ResNet. I believe this is the case since the CIFAR10 dataset, though leagues more complex than the MNIST dataset, is not complex enough to require the extraction, finetuning, and filtering of those features over a long range. Instead, blowing up the number of features immediately and then finding more and more features as fast as possible seems to be a better option, as can be seen with the CNN model architecture.

In conclusion, the CNN model from the first half of this homework is better suited for CIFAR10 Image Classification.


## Code Used

### Imports and Data Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils import data
from tqdm import tqdm

In [ ]:
# Grab the MNIST dataset
training_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testing_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

tfm = transforms.Compose([
    transforms.ToTensor(),
])

trainset_full_CIFAR10 = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=tfm)
testset_full_CIFAR10  = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=tfm)

In [ ]:
# Verify that GPU is connected and available

print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.cuda.get_device_name(0))

### CNN Classifier

In [ ]:
class CIFAR10_Classifier(nn.Module):
    def __init__(self, C_dropout, F_dropout):
        super().__init__()

        conv2d_dropout = C_dropout

        conv_layer_1 = 256
        conv_layer_2 = 256

        conv_layer_3 = 256
        conv_layer_4 = 256

        self.forward_funnel_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=conv_layer_1, kernel_size=3),   # Extract useful features from the beginning
            nn.BatchNorm2d(num_features=conv_layer_1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(conv2d_dropout),

            nn.Conv2d(in_channels=conv_layer_1, out_channels=conv_layer_2, kernel_size=3),  # Extract useful features from the learned features
            nn.BatchNorm2d(num_features=conv_layer_2),
            nn.ReLU(inplace=True),
            nn.Dropout2d(conv2d_dropout),
            nn.MaxPool2d(kernel_size=2, stride=2),                       # Reduce dimensionality
        )

        self.forward_funnel_2 = nn.Sequential(
            nn.Conv2d(in_channels=conv_layer_2, out_channels=conv_layer_3, kernel_size=3),   # Extract useful features from the beginning
            nn.BatchNorm2d(num_features=conv_layer_3),
            nn.ReLU(inplace=True),
            nn.Dropout2d(conv2d_dropout),

            nn.Conv2d(in_channels=conv_layer_3, out_channels=conv_layer_4, kernel_size=3),  # Extract useful features from the learned features
            nn.BatchNorm2d(num_features=conv_layer_4),
            nn.ReLU(inplace=True),
            nn.Dropout2d(conv2d_dropout),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Compute the number of features after the input has passed the funnel
        with torch.no_grad():
            test_input = torch.zeros(1, 3, 32, 32)

            test_input.to(device)

            features = self.forward_funnel_1(test_input)
            features = self.forward_funnel_2(features)

            total_count = features.view(1, -1).size(1)

        full_node_dropout = F_dropout


        lin_layer_1_size = 1000
        lin_layer_2_size = 500
        lin_layer_3_size = 250

        self.output_nodes = 100

        self.classifer = nn.Sequential(
            nn.Flatten(),                                           # Flatten the image from the funnel
            nn.Linear(in_features=total_count, out_features=lin_layer_1_size),
            nn.LayerNorm(lin_layer_1_size),
            nn.ReLU(inplace=True),
            nn.Dropout(full_node_dropout),

            nn.Linear(in_features=lin_layer_1_size, out_features=lin_layer_2_size),
            nn.LayerNorm(lin_layer_2_size),
            nn.ReLU(inplace=True),
            nn.Dropout(full_node_dropout),

            nn.Linear(in_features=lin_layer_2_size, out_features=lin_layer_3_size),
            nn.LayerNorm(lin_layer_3_size),
            nn.ReLU(inplace=True),
            nn.Dropout(full_node_dropout),

            nn.Linear(in_features=lin_layer_3_size, out_features=self.output_nodes),
            nn.LayerNorm(self.output_nodes),
            nn.ReLU(inplace=True),
            nn.Dropout(full_node_dropout),
        )

        self.output_layer = nn.Linear(in_features=self.output_nodes, out_features=10)

    def partial_forward(self, x):
        x = self.forward_funnel_1(x)
        x = self.forward_funnel_2(x)
        x = self.classifer(x)

        return x

    def forward(self, x):
        x = self.partial_forward(x)
        logits = self.output_layer(x)

        return logits

### Data Collection and CNN Model Training Loop

In [ ]:
epoch_over_training_loss_CIFAR10 = []
epoch_over_testing_loss_CIFAR10 = []

In [ ]:
# Hyperparameter setup
epochs = 31
batch_size = 64
learning_rate = 1e-4
decay_rate = 4e-4

c_dropout = 0.30
f_dropout = 0.30

print('######## Begining training for CIFAR10 classifier ##########')

# Setup data loaders
trainset_loader_CIFAR10 = data.DataLoader(trainset_full_CIFAR10,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   num_workers=8,
                                   pin_memory=True)

testset_loader_CIFAR10 = data.DataLoader(testset_full_CIFAR10,
                                   batch_size=batch_size,
                                   num_workers=8,
                                   shuffle=False,
                                   pin_memory=True)

model = CIFAR10_Classifier(c_dropout, f_dropout)
model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),
                       lr=learning_rate,
                       weight_decay=decay_rate
                       )

# Have references to variables outside of the epoch loop
avg_training_loss = 0
avg_testing_loss = 0

# Epoch Loop
for epoch in range(epochs):
    print(f'----- Epoch: {epoch + 1}/{epochs} -----')

    avg_training_loss = 0
    avg_testing_loss = 0

    model.train()

    for x, Y in tqdm(trainset_loader_CIFAR10, desc='Training', unit=' batch'):
        # Transfer images to GPU
        x = x.to(device)
        Y = Y.to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Send images to model
        x_pred = model(x)

        # Calc loss
        loss = loss_function(x_pred, Y)

        # Calc gradient and update weights
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            avg_training_loss += loss.item()

    # Switch to eval mode
    model.eval()

    with torch.no_grad():
        for x, Y in tqdm(testset_loader_CIFAR10, desc='Testing', unit=' batches'):
            # Move the images to the GPU
            x = x.to(device)
            Y = Y.to(device)

            # Get logits and sum up total loss
            x_pred = model(x)
            avg_testing_loss += loss_function(x_pred, Y).item()

    # Get training loss
    avg_training_loss /= len(trainset_loader_CIFAR10)

     # Get testing loss
    avg_testing_loss /= len(testset_loader_CIFAR10)

    # Switch model back to training mode
    model.train()

    epoch_over_training_loss_CIFAR10.append({
        "epoch": epoch,
        "training_loss": avg_training_loss
        })

    epoch_over_testing_loss_CIFAR10.append({
        "epoch": epoch,
        "testing_loss": avg_testing_loss
        })


    print("")

    print(f'   -> Training Loss: {avg_training_loss: .4f}\n')
    print(f'   -> Testing Loss: {avg_testing_loss: .4f}\n')


### CNN Model Performance Data Collection

In [ ]:
model.eval()

correct = total = 0

with torch.no_grad():
    for xb, yb in testset_loader_CIFAR10:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        total += yb.numel()

final_accuracy = float(correct / total)

print(f"Final Accuracy: {final_accuracy}")


train_loss = [d["training_loss"] for d in epoch_over_training_loss_CIFAR10]
epochs = range(1, len(train_loss) + 1)
test_loss = [d["testing_loss"] for d in epoch_over_testing_loss_CIFAR10]

plt.figure(figsize=(8, 5))
plt.plot(epochs, train_loss, label="Training Loss")
plt.plot(epochs, test_loss, label="Testing Loss")

plt.xlim(0.9, len(train_loss) + 0.1)

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("CIFAR10 CNN Classifer: Training and Testing Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()

### ResNet Model

In [ ]:
class CNN_Block(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super().__init__()

        # self.cnn_dropout = cnn_dropout

        self.skip = nn.Sequential()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=in_planes, out_channels=planes, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(planes),

            nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(planes),
        )

        if stride != 1 or in_planes != planes:
            # Use a 1x1 convolution to match dimensions
            self.skip = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):

        # Pass the input through the block
        logits = self.conv_block(x)

        # Skip the original data
        logits += self.skip(x)

        # Activation Function
        logits = F.relu(logits)

        return logits


class CIFAR10_ResNet(nn.Module):
    def __init__(self, num_blocks:list, num_classes=10, linear_dropout=0.25):
        super().__init__()

        # Initial size of the CNN layer that accepts the image
        # Also used when creating stages of blocks self.stage_layer
        self.in_planes = 32

        self.image_input_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=self.in_planes, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.in_planes),
            nn.ReLU(inplace=True)
        )

        # range_beginning = 48
        # range_end = 144
        # range_step = range_beginning
        #
        # self.cnn_plane_range = range(range_beginning, range_end + 1, range_step)

        # self.stages = nn.Sequential(
        #     *[self.make_stage(planes, num_block, 1 if idx == 0 else 2)
        #         for idx, (planes, num_block) in enumerate(zip(self.cnn_plane_range, num_blocks))]
        # )

        self.stages = nn.Sequential(
            self.make_stage(32, num_blocks[0], 1),
            self.make_stage(64, num_blocks[1], 2),
            self.make_stage(96, num_blocks[2], 2),
        )

        self.lin_layer_1_size = 2000
        self.lin_layer_2_size = 1500
        self.lin_layer_3_size = 1000
        self.lin_layer_4_size = 500


        self.classifier = nn.Sequential(
            nn.Linear(96, self.lin_layer_1_size),
            # nn.LayerNorm(self.lin_layer_1_size),
            nn.ReLU(inplace=True),
            nn.Dropout(linear_dropout),

            nn.Linear(self.lin_layer_1_size, self.lin_layer_2_size),
            # nn.LayerNorm(self.lin_layer_2_size),
            nn.ReLU(inplace=True),
            nn.Dropout(linear_dropout),

            nn.Linear(self.lin_layer_2_size, self.lin_layer_3_size),
            # nn.LayerNorm(self.lin_layer_3_size),
            nn.ReLU(inplace=True),
            nn.Dropout(linear_dropout),

            nn.Linear(self.lin_layer_3_size, self.lin_layer_4_size),
            nn.LayerNorm(self.lin_layer_4_size),
            nn.ReLU(inplace=True),
            nn.Dropout(linear_dropout),

            nn.Linear(self.lin_layer_4_size, num_classes)
        )


    def make_stage(self, planes, num_blocks, stride):

        strides = [stride] + [1] * (num_blocks - 1)

        layers = []

        for stride in strides:
            # Add ResBlock to list
            layers.append(CNN_Block(self.in_planes, planes, stride))
            # Reset the in planes to preserve in_channels of the next blocks
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):

        logits = self.image_input_layer(x)

        logits = self.stages(logits)

        logits = F.avg_pool2d(logits, 8)

        logits = logits.view(logits.size(0), -1)

        logits = self.classifier(logits)

        return logits



### Data Setup and ResNet Model Training Loop

In [ ]:
epoch_over_training_loss_CIFAR10 = []
epoch_over_testing_loss_CIFAR10 = []

# Hyperparameter setup
epochs = 100
batch_size = 64
learning_rate = 1e-3
decay_rate = 5e-4

f_dropout = 0.50

print('######## Beginning training for CIFAR10 ResNet classifier ##########')

# Setup data loaders
trainset_loader_CIFAR10 = data.DataLoader(trainset_full_CIFAR10,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   num_workers=8,
                                   pin_memory=True)

testset_loader_CIFAR10 = data.DataLoader(testset_full_CIFAR10,
                                   batch_size=batch_size,
                                   num_workers=8,
                                   shuffle=False,
                                   pin_memory=True)

model = CIFAR10_ResNet(num_blocks=[3, 3, 3], num_classes=10, linear_dropout=f_dropout)
model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),
                       lr=learning_rate,
                       weight_decay=decay_rate
                       )

# Have references to variables outside of the epoch loop
avg_training_loss = 0
avg_testing_loss = 0

# Epoch Loop
for epoch in range(epochs):
    print(f'----- Epoch: {epoch + 1}/{epochs} -----')

    avg_training_loss = 0
    avg_testing_loss = 0

    model.train()

    for x, Y in tqdm(trainset_loader_CIFAR10, desc='Training', unit=' batch'):
        # Transfer images to GPU
        x = x.to(device)
        Y = Y.to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Send images to model
        x_pred = model(x)

        # Calc loss
        loss = loss_function(x_pred, Y)

        # Calc gradient and update weights
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            avg_training_loss += loss.item()

    # Switch to eval mode
    model.eval()

    with torch.no_grad():
        for x, Y in tqdm(testset_loader_CIFAR10, desc='Testing', unit=' batches'):
            # Move the images to the GPU
            x = x.to(device)
            Y = Y.to(device)

            # Get logits and sum up total loss
            x_pred = model(x)
            avg_testing_loss += loss_function(x_pred, Y).item()

    # Get training loss
    avg_training_loss /= len(trainset_loader_CIFAR10)

     # Get testing loss
    avg_testing_loss /= len(testset_loader_CIFAR10)

    # Switch model back to training mode
    model.train()

    epoch_over_training_loss_CIFAR10.append({
        "epoch": epoch,
        "training_loss": avg_training_loss
        })

    epoch_over_testing_loss_CIFAR10.append({
        "epoch": epoch,
        "testing_loss": avg_testing_loss
        })


    print("")

    print(f'   -> Training Loss: {avg_training_loss: .4f}\n')
    print(f'   -> Testing Loss: {avg_testing_loss: .4f}\n')


### ResNet Model Performance Data Collection

In [ ]:
model.eval()

correct = total = 0

with torch.no_grad():
    for xb, yb in testset_loader_CIFAR10:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        pred = logits.argmax(1)
        correct += (pred == yb).sum().item()
        total += yb.numel()

final_accuracy = float(correct / total)

print(f"Final Accuracy: {final_accuracy}")


train_loss = [d["training_loss"] for d in epoch_over_training_loss_CIFAR10]
epochs = range(1, len(train_loss) + 1)
test_loss = [d["testing_loss"] for d in epoch_over_testing_loss_CIFAR10]

plt.figure(figsize=(8, 5))
plt.plot(epochs, train_loss, label="Training Loss")
plt.plot(epochs, test_loss, label="Testing Loss")

plt.xlim(0.9, len(train_loss) + 0.1)

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("CIFAR10 ResNet Classifer: Training and Testing Loss per Epoch")
plt.legend()
plt.grid(True)
plt.show()